Churn prediction and Customer segmentation with balanced data

In [65]:
import pandas as pd
import numpy as np
from sklearn.utils import resample


from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn import svm

from scipy.stats import pearsonr, spearmanr

from matplotlib import pyplot as plt
from sklearn.utils import resample


from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

In [66]:
data = pd.read_csv('Churn_Modelling.csv')
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9997,9998,15584532,Liu,709,France,Female,36.0,7,0.00,1,0.0,1.0,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
9999,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
10000,10000,15628319,Walker,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0


In [67]:
data = data.drop(['Surname', 'RowNumber', 'CustomerId'], axis=1) 
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9997,709,France,Female,36.0,7,0.00,1,0.0,1.0,42085.58,1
9998,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
9999,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
10000,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0


In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10002 non-null  int64  
 1   Geography        10001 non-null  object 
 2   Gender           10002 non-null  object 
 3   Age              10001 non-null  float64
 4   Tenure           10002 non-null  int64  
 5   Balance          10002 non-null  float64
 6   NumOfProducts    10002 non-null  int64  
 7   HasCrCard        10001 non-null  float64
 8   IsActiveMember   10001 non-null  float64
 9   EstimatedSalary  10002 non-null  float64
 10  Exited           10002 non-null  int64  
dtypes: float64(5), int64(4), object(2)
memory usage: 859.7+ KB


In [69]:
data.shape

(10002, 11)

In [70]:
data.dropna(inplace=True)
data.drop_duplicates(keep='first', inplace = True)

In [71]:
data.shape
data['Exited'].value_counts()

0    7959
1    2037
Name: Exited, dtype: int64

In [72]:
data.Geography = data.Geography.map({'France': 0, 'Germany': 1, 'Spain': 2})
data.Gender = data.Gender.map({'Female': 0, 'Male': 1})

In [73]:
majority_class_data = data[data['Exited'] == 0]
minority_class_data = data[data['Exited'] == 1]

# Upsample the minority class to balance the dataset
upsampled_minority_class = resample(minority_class_data, replace=True, n_samples=len(majority_class_data))
# Combine the upsampled minority class with the majority class
data = pd.concat([majority_class_data, upsampled_minority_class], axis=0)

data.shape


(15918, 11)

In [74]:
y = data.Exited
data.drop(columns = ['Exited'], inplace = True)

In [75]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size = 0.3)

In [76]:
list_of_models = {
    'logistic_regression' : LogisticRegression(random_state = 5, max_iter = 1000),
    'decision_tree' : DecisionTreeClassifier(max_depth = 5, random_state = 42,criterion='entropy'),
    'kneighbirs' : KNeighborsClassifier(n_neighbors=3),
    'Gaussian' : GaussianNB(),
    "svm" : svm.SVC(kernel='rbf') 
}

In [77]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [79]:
accuracy_s = []
f1_train_scores = [] 
f1_test_scores = [] 
recall_test_scores = []

model_names = list_of_models.keys()

for model in model_names:
    
    list_of_models[model].fit(X_train, y_train)
    y_train_pred = list_of_models[model].predict(X_train)
    

    f1_train = f1_score(y_train, y_train_pred, average='macro')

    y_test_pred = list_of_models[model].predict(X_test)
    
    
    f1_test = f1_score(y_test, y_test_pred, average='macro')

    recall_test_score = recall_score(y_test, y_test_pred, average='macro')
    
    accuracy_s.append(accuracy_score(y_test, y_test_pred))
    f1_train_scores.append(f1_train)
    f1_test_scores.append(f1_test)
    recall_test_scores.append(recall_test_score)


d = {
    'accuracy_s': accuracy_s,
    'f1_train_scores':f1_train_scores,
    'f1 test_scores': f1_test_scores,
    'Recall test scores': recall_test_scores
}
df = pd.DataFrame(d)
df.index = ['logistic_regression','decision_tree' ,'kneighbors' ,'Gaussian','svm']
print(df)

 


                     accuracy_s  f1_train_scores  f1 test_scores  \
logistic_regression    0.701424         0.704427        0.701380   
decision_tree          0.755444         0.767769        0.754648   
kneighbors             0.848409         0.925362        0.847637   
Gaussian               0.728224         0.733740        0.727812   
svm                    0.794179         0.813767        0.794167   

                     Recall test scores  
logistic_regression            0.701375  
decision_tree                  0.755116  
kneighbors                     0.848889  
Gaussian                       0.728009  
svm                            0.794162  


In [86]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create a simple neural network model
model = Sequential([Dense(10, activation='sigmoid'), Dense(1, activation='sigmoid')])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

y_train_pred = model.predict(X_train)



Epoch 1/10
279/279 [==============================] - 1s 2ms/step - loss: 0.7015 - accuracy: 0.5613 - val_loss: 0.6544 - val_accuracy: 0.6738
Epoch 2/10
279/279 [==============================] - 1s 2ms/step - loss: 0.6353 - accuracy: 0.6894 - val_loss: 0.6154 - val_accuracy: 0.6994
Epoch 3/10
279/279 [==============================] - 1s 2ms/step - loss: 0.6057 - accuracy: 0.7012 - val_loss: 0.5906 - val_accuracy: 0.7120
Epoch 4/10
279/279 [==============================] - 1s 2ms/step - loss: 0.5887 - accuracy: 0.7010 - val_loss: 0.5769 - val_accuracy: 0.7093
Epoch 5/10
279/279 [==============================] - 1s 3ms/step - loss: 0.5800 - accuracy: 0.7044 - val_loss: 0.5698 - val_accuracy: 0.7133
Epoch 6/10
279/279 [==============================] - 1s 2ms/step - loss: 0.5751 - accuracy: 0.7066 - val_loss: 0.5652 - val_accuracy: 0.7178
Epoch 7/10
279/279 [==============================] - 1s 2ms/step - loss: 0.5715 - accuracy: 0.7106 - val_loss: 0.5613 - val_accuracy: 0.7236
Epoch 

In [90]:
# Create a simple neural network model
model = Sequential([Dense(100, activation='relu'), Dense(10, activation='relu'),Dense(1, activation='relu')])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

y_train_pred = model.predict(X_train)


Epoch 1/10
279/279 [==============================] - 2s 3ms/step - loss: 0.9913 - accuracy: 0.6261 - val_loss: 0.6582 - val_accuracy: 0.6828
Epoch 2/10
279/279 [==============================] - 1s 2ms/step - loss: 0.6525 - accuracy: 0.6995 - val_loss: 0.6860 - val_accuracy: 0.6891
Epoch 3/10
279/279 [==============================] - 1s 2ms/step - loss: 0.5712 - accuracy: 0.7259 - val_loss: 0.5513 - val_accuracy: 0.7465
Epoch 4/10
279/279 [==============================] - 1s 2ms/step - loss: 0.5273 - accuracy: 0.7478 - val_loss: 0.5638 - val_accuracy: 0.7564
Epoch 5/10
279/279 [==============================] - 1s 2ms/step - loss: 0.6782 - accuracy: 0.7201 - val_loss: 0.5809 - val_accuracy: 0.7115
Epoch 6/10
279/279 [==============================] - 1s 2ms/step - loss: 0.5264 - accuracy: 0.7534 - val_loss: 0.5416 - val_accuracy: 0.7586
Epoch 7/10
279/279 [==============================] - 1s 2ms/step - loss: 0.6568 - accuracy: 0.7361 - val_loss: 0.5427 - val_accuracy: 0.7008
Epoch 

In [97]:

# Create a simple neural network model
model = Sequential([Dense(150, activation='sigmoid'), Dense(100, activation='relu'),Dense(10, activation='sigmoid'),Dense(1, activation='relu')])
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

y_train_pred = model.predict(X_train)

Epoch 1/10
279/279 [==============================] - 2s 3ms/step - loss: 7.7151 - accuracy: 0.4998 - val_loss: 7.8059 - val_accuracy: 0.4939
Epoch 2/10
279/279 [==============================] - 1s 2ms/step - loss: 7.7151 - accuracy: 0.4998 - val_loss: 7.8059 - val_accuracy: 0.4939
Epoch 3/10
279/279 [==============================] - 1s 2ms/step - loss: 7.7151 - accuracy: 0.4998 - val_loss: 7.8059 - val_accuracy: 0.4939
Epoch 4/10
279/279 [==============================] - 1s 3ms/step - loss: 7.7151 - accuracy: 0.4998 - val_loss: 7.8059 - val_accuracy: 0.4939
Epoch 5/10
279/279 [==============================] - 1s 2ms/step - loss: 7.7151 - accuracy: 0.4998 - val_loss: 7.8059 - val_accuracy: 0.4939
Epoch 6/10
279/279 [==============================] - 1s 2ms/step - loss: 7.7151 - accuracy: 0.4998 - val_loss: 7.8059 - val_accuracy: 0.4939
Epoch 7/10
279/279 [==============================] - 1s 2ms/step - loss: 7.7151 - accuracy: 0.4998 - val_loss: 7.8059 - val_accuracy: 0.4939
Epoch 